<a href="https://colab.research.google.com/github/SebastianLealT/OPTIMIZACION-1/blob/main/Optimizaci%C3%B3n_en_varias_variables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#*Métodos de optimización: Funciones de varias variables*
##Johan Sebastian Leal Toro
##Curso de Optimización I
##Universidad del Valle, Cali

En este cuaderno nos centraremos en estudiar los siguientes métodos:
1. Método de Nelder-Mead
2. 
3. 
4. 
